## Twitter data given user id

Some comment: it looks like that using this API, you don' t need the dev environment and you can download 2kk tweets a month. 

## Set up

In [2]:
import os
os.environ['TOKEN'] ='' #your bearer token here

### Import libraries

In [4]:
import requests 
import pandas as pd 
import time

### Set up headers

In [5]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [6]:
headers = create_headers(os.environ['TOKEN'])

## Download tweet from a specific id in a time window

to get the id given the "@name", you can use : 
https://codeofaninja.com/tools/find-twitter-id/



In [32]:
def create_url(id, start_date, end_date):
    
    # Timeline by id
    search_url = "https://api.twitter.com/2/users/{}/tweets".format(id)

    #change params based on the endpoint you are using
    query_params = {'max_results': 5, 'start_time': start_date, 'end_time': end_date}
    return (search_url, query_params)

In [17]:
def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token is not None and next_token != '':
        params['next'] = next_token
    response = requests.request("GET", url, headers = headers, params = params,)
    print(response.url) # for debug
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [19]:
def get_data(id, start_time, end_time, next_token):   ##NOT TESTED
    results = []
  
    while next_token is not None:
        ##this part here for one request
        url = create_url(id, start_time,end_time)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)

        if "data" in json_response:
            results.extend(json_response["data"])
        ### up until this point
        if "next_token" in json_response['meta']:
            next_token = json_response["meta"]["next_token"]
        else:
            next_token = None
        time.sleep(1.1)
  
  return results

In [58]:
def get_single_response(id, start_time, end_time):
    results = []
    url = create_url(id, start_time,end_time)
    json_response = connect_to_endpoint(url[0], headers, url[1])

    #print(json_response)
    if "data" in json_response:
        results.extend(json_response["data"])

    return results


In [59]:
# example for @DPCGov between  the 12 and 19 december 2019
tweets = get_single_response("982240377636700161", "2019-12-12T00:00:00Z", "2019-12-19T00:00:00Z")

https://api.twitter.com/2/users/982240377636700161/tweets?max_results=5&start_time=2019-12-12T00%3A00%3A00Z&end_time=2019-12-19T00%3A00%3A00Z


### Inspect data

In [60]:
tweets

[{'id': '1207339601049899013',
  'text': 'RT @Quirinale: #Mattarella: Profonda riconoscenza esprimo alle amministrazioni, ai Servizi e ai Corpi dello Stato. Dai #VigilidelFuoco alla…'},
 {'id': '1207328639391555585',
  'text': '🔔#allertaARANCIONE, giovedì #19dicembre, su parte della Sicilia. \n🔔#allertaGIALLA in 5 regioni.\n\n🌧🌬 Avviso di condizioni meteorologiche avverse del #18dicembre per piogge e temporali su Sicilia, Calabria e Campania. Leggi qui 👉 https://t.co/J2y2LtNY5U\n#protezionecivile https://t.co/ylJm64gNtS'},
 {'id': '1207279054333841416',
  'text': "#18dicembre Chiusura numero solidale: raccolti 574.114,00 euro.\nGrazie alla generosità di chi ha voluto sostenere la città di #Venezia gravemente colpita dall'eccezionale evento di acqua alta lo scorso novembre 👉 https://t.co/CScRy2siXs https://t.co/GjzuEdXQ79"},
 {'id': '1207208205492916224',
  'text': 'Le domande che spopolano in questo periodo:\nChe fai a Capodanno?🤷\u200d♀️\nChe tempo farà a Capodanno? 👇👇\n#18dicembre #p

In [61]:
tweets[0]

{'id': '1207339601049899013',
 'text': 'RT @Quirinale: #Mattarella: Profonda riconoscenza esprimo alle amministrazioni, ai Servizi e ai Corpi dello Stato. Dai #VigilidelFuoco alla…'}